In [ ]:
#| default_exp utils

In [ ]:
#| hide
%load_ext autoreload
%autoreload 2

# Utils

> Utilities used in the rest of the notebooks

In [ ]:
#| export
from dvats.imports import *
from fastcore.all import *
import wandb
import pickle
import pandas as pd
import numpy as np
#import tensorflow as tf
import torch.nn as nn
from fastai.basics import *

### Generate random time series dataframe

In [ ]:
#| export
def generate_TS_df(rows, cols):
    "Generates a dataframe containing a multivariate time series, where each column \
    represents a variable and each row a time point (sample). The timestamp is in the \
    index of the dataframe, and it is created with a even space of 1 second between samples"
    index = np.arange(pd.Timestamp.now(),
                      pd.Timestamp.now() + pd.Timedelta(rows-1, 'seconds'),
                      pd.Timedelta(1, 'seconds'))
    data = np.random.randn(len(index), cols)
    return pd.DataFrame(data, index=index)

In [ ]:
df = generate_TS_df(3, 5)

In [ ]:
test_eq(df.shape, (3, 5))

##  pandas Dataframe utilities

### Normalize columns

In [ ]:
#| export
def normalize_columns(df:pd.DataFrame):
    "Normalize columns from `df` to have 0 mean and 1 standard deviation"
    mean = df.mean()
    std = df.std() + 1e-7
    return (df-mean)/std

In [ ]:
foo = generate_TS_df(3, 3)
foo.describe()

0         1         2
count  3.000000  3.000000  3.000000
mean  -0.356446 -0.217669 -0.451268
std    1.567722  0.268632  0.309269
min   -1.327362 -0.480470 -0.693929
25%   -1.260753 -0.354722 -0.625384
50%   -1.194144 -0.228973 -0.556840
75%    0.129012 -0.086268 -0.329938
max    1.452169  0.056437 -0.103037

In [ ]:
bar = normalize_columns(foo)
bar.describe()

0             1             2
count  3.000000e+00  3.000000e+00  3.000000e+00
mean  -3.700743e-17 -3.700743e-17 -9.251859e-17
std    9.999999e-01  9.999996e-01  9.999997e-01
min   -6.193166e-01 -9.782957e-01 -7.846253e-01
25%   -5.768289e-01 -5.101875e-01 -5.629913e-01
50%   -5.343411e-01 -4.207935e-02 -3.413573e-01
75%    3.096583e-01  4.891479e-01  3.923127e-01
max    1.153658e+00  1.020375e+00  1.125983e+00

In [ ]:
test_close(bar.describe().loc['mean'].values, np.repeat(0.0, len(bar.columns)))

In [ ]:
test_close(bar.describe().loc['std'].values, np.repeat(1.0, len(bar.columns)))

### Remove constant columns

In [ ]:
#| export
def remove_constant_columns(df:pd.DataFrame):
    return df.loc[:, (df != df.iloc[0]).any()]

In [ ]:
foo = generate_TS_df(3, 3)
foo['constant'] = [0.0]*len(foo)
foo

0         1         2  constant
2022-04-23 10:32:47.134188 -0.658749  1.680371  1.426223       0.0
2022-04-23 10:32:48.134188  1.961724  1.247468 -1.358013       0.0
2022-04-23 10:32:49.134188 -0.195324  0.413195  0.504546       0.0

In [ ]:
bar = remove_constant_columns(foo)
bar

0         1         2
2022-04-23 10:32:47.134188 -0.658749  1.680371  1.426223
2022-04-23 10:32:48.134188  1.961724  1.247468 -1.358013
2022-04-23 10:32:49.134188 -0.195324  0.413195  0.504546

In [ ]:
column_diff = set(foo.columns) - set(bar.columns)
test_eq_type(column_diff, set(['constant']))

## Create wandb artifact containing just the reference to an object pass as argument

In [ ]:
#| export
class ReferenceArtifact(wandb.Artifact):
    default_storage_path = Path('data/wandb_artifacts/') # * this path is relative to Path.home()
    "This class is meant to create an artifact with a single reference to an object \
    passed as argument in the contructor. The object will be pickled, hashed and stored \
    in a specified folder."
    @delegates(wandb.Artifact.__init__)
    def __init__(self, obj, name, type='object', folder=None, **kwargs):
        super().__init__(type=type, name=name, **kwargs)
        # pickle dumps the object and then hash it
        hash_code = str(hash(pickle.dumps(obj)))
        folder = Path(ifnone(folder, Path.home()/self.default_storage_path))
        with open(f'{folder}/{hash_code}', 'wb') as f:
            pickle.dump(obj, f)
        self.add_reference(f'file://{folder}/{hash_code}')
        if self.metadata is None:
            self.metadata = dict()
        self.metadata['ref'] = dict()
        self.metadata['ref']['hash'] = hash_code
        self.metadata['ref']['type'] = str(obj.__class__)

In [ ]:
foo = np.arange(10)
bar = ReferenceArtifact(obj=foo, name='foo', folder='.')
bar_path = Path(f'./{bar.metadata["ref"]["hash"]}')
test_eq(bar_path.exists(), True)
test_eq(bar.metadata['ref']['type'], "<class 'numpy.ndarray'>")

When a reference artifact is used by one wandb run, we should have a method to get the original object from it

In [ ]:
#| export
@patch
def to_obj(self:wandb.apis.public.Artifact):
    """Download the files of a saved ReferenceArtifact and get the referenced object. The artifact must \
    come from a call to `run.use_artifact` with a proper wandb run."""
    if self.metadata.get('ref') is None:
        print(f'ERROR:{self} does not come from a saved ReferenceArtifact')
        return None
    original_path = ReferenceArtifact.default_storage_path/self.metadata['ref']['hash']
    path = original_path if original_path.exists() else Path(self.download()).ls()[0]
    with open(path, 'rb') as f:
        obj = pickle.load(f)
    return obj

Test with Reference artifact from a df

In [ ]:
foo = generate_TS_df(3, 3)
bar = ReferenceArtifact(obj=foo, name='test_reference_artifact')
bar.manifest.entries.values()

dict_values([<ManifestEntry ref: file:///home/dmontalvo/data/wandb_artifacts/776758047213944476/776758047213944476>])

In [ ]:
test_eq(bar.name, 'test_reference_artifact')

In [ ]:
test_eq(bar.metadata['ref']['type'], str(type(foo)))

TODO: Test method `to_obj`

ReferenceArtifact with a numpy array

In [ ]:
foo = np.random.randn(5)
bar = ReferenceArtifact(obj=foo, name='test_reference_artifact')
bar.manifest.entries.values()

dict_values([<ManifestEntry ref: file:///home/dmontalvo/data/wandb_artifacts/-8161467246553937997/-8161467246553937997>])

In [ ]:
test_eq(bar.metadata['ref']['type'], str(type(foo)))

In [ ]:
#| export
import torch.nn as nn
class PrintLayer(nn.Module):
    def __init__(self):
        super(PrintLayer, self).__init__()

    def forward(self, x):
        # Do your print / debug stuff here
        print(x.shape)
        return x

In [ ]:
#| export
@patch
def export_and_get(self:Learner, keep_exported_file=False):
    """
        Export the learner into an auxiliary file, load it and return it back.
    """
    aux_path = Path('aux.pkl')
    self.export(fname='aux.pkl')
    aux_learn = load_learner('aux.pkl')
    if not keep_exported_file: aux_path.unlink()
    return aux_learn

### get_wandb_artifacts

In [ ]:
#| export
def get_wandb_artifacts(project_path, type=None, name=None, last_version=True):
    """
        Get the artifacts logged in a wandb project.
        Input:
        - `project_path` (str): entity/project_name
        - `type` (str): whether to return only one type of artifacts
        - `name` (str): Leave none to have all artifact names
        - `last_version`: whether to return only the last version of each artifact or not

        Output: List of artifacts
    """
    public_api = wandb.Api()
    if type is not None:
        types = [public_api.artifact_type(type, project_path)]
    else:
        types = public_api.artifact_types(project_path)

    res = L()
    for kind in types:
        for collection in kind.collections():
            if name is None or name == collection.name:
                versions = public_api.artifact_versions(
                    kind.type,
                    "/".join([kind.entity, kind.project, collection.name]),
                    per_page=1,
                )
                if last_version: res += next(versions)
                else: res += L(versions)
    return list(res)

In [ ]:
foo = get_wandb_artifacts('wandb/artifacts-example', type='model')
test_eq(len(foo), 2)
foo = get_wandb_artifacts('wandb/artifacts-example', type='model', name='convnet')
test_eq(len(foo), 1)
foo = get_wandb_artifacts('wandb/artifacts-example', type='model', name='convnet', last_version=False)
test_eq(len(foo), 2)

### get_pickle_artifact

In [ ]:
#| export
def get_pickle_artifact(filename):

    with open(filename, "rb") as f:
        df = pickle.load(f)
    
    return df

## Pickled
Functions execution loading/saving input/output as pickled objects

In [ ]:
#| export
def pickled_file(function, filename=None, hash_code = None, include_hash_code = True):
    filename_is_none = filename is None
    if filename_is_none:
        filename = f"result_{function.__name__}"
    if not hash_code is None and (include_hash_code or filename_is_none):
        filename = f"{filename}_{hash_code}"
    filename = f"{filename}.pickle"
    return filename

In [ ]:
#| hide
filename = None
hash_code = None
def foo(a,b,c=1): 
    print(f"({a}+{b})*{c}")
    return (a+b)*c
    
print(f"{filename} | {hash_code}")
print(pickled_file(foo, filename, hash_code))

hash_code = 1
print(f"{filename} | {hash_code}")
print(pickled_file(foo,filename, hash_code))

filename = "Potato"
print(f"{filename} | {hash_code}")
print(pickled_file(foo, filename, hash_code))

print(f"{filename} | {hash_code}")
print(pickled_file(foo, filename, hash_code, False))

filename = None
print(f"{filename} | {hash_code}")
print(pickled_file(foo, filename, hash_code, False))

None | None
result_foo.pickle
None | 1
result_foo_1.pickle
Potato | 1
Potato_1.pickle
Potato | 1
Potato.pickle
None | 1
result_foo_1.pickle


In [ ]:
def exec_and_pickle(function, filename=None, *args, **kwargs):
    result = function(*args, **kwargs)
    hash_code = str(hash(pickle.dumps(result)))
    filename = pickled_file(function, filename, hash_code)
    with open(filename, 'wb') as f:
        pickle.dump(result,f)
    return filename, hash_code

In [ ]:
print(foo(1,2))
filename, hash_code = exec_and_pickle(foo, None, 1, 2)
print(filename)
print(hash_code)

(1+2)*1
3
(1+2)*1
result_foo_6814786563595406894.pickle
6814786563595406894


In [ ]:
def exec_with_pickle(function, filename = "", hash_code = None, include_hash_code = True, print_flag=False, *args, **kwargs): 
    filename = pickled_file(function, filename, hash_code, include_hash_code)
    print(filename)
    with open(filename, 'rb') as f:
        input = pickle.load(f)
    if print_flag: 
        print( f"input: {input}, args: {args}, kwargs: {kwargs}")
    result = function(input, *args, **kwargs) 
    return result

In [ ]:
exec_with_pickle(foo, None, hash_code, True, True, 4, 2)

result_foo_6814786563595406894.pickle
input: 3, args: (4, 2), kwargs: {}
(3+4)*2


14

In [ ]:
import os
def remove_pickled(function, filename = None, hash_code = None, include_hash_code = True, print_flag = False):
    filename = pickled_file(function, filename, hash_code, include_hash_code)
    try:
        os.remove(filename)
        if print_flag: print(f"Filename {filename} deleted")
    except FileNotFoundError:
        print(f"File {filename} not found")
    except Exception as e: 
        print(f"Error deleting {filename}: {e}")

In [ ]:
remove_pickled(foo, None, hash_code, True, True)

Filename result_foo_6814786563595406894.pickle deleted


## Export

In [ ]:
#| hide
#from nbdev.export import notebook2script
#notebook2script()
beep(1)

Converted dr.ipynb.
Converted encoder.ipynb.
Converted index.ipynb.
Converted load.ipynb.
Converted utils.ipynb.
Converted visualization.ipynb.
